In [1]:
import json
import os
os.chdir("..")

In [2]:
import pprint

In [3]:
from IO.constants import Constants

In [4]:
# Inputs

path = os.path.join(os.getcwd(), "utils", "799988a6cbeb", "Input.registry.file")
with open(path, "r") as file:
    input_config_file = json.loads(file.read())

input_config_mqtt = {}
model_name = "CarParkModel"

## CarPark

In [39]:
class CarPark:

    def __init__(self, chargers_list, cars_list):

        self.cars = {}
        self.chargers = {charger_index: charger for charger_index, charger in enumerate(chargers_list)}
        self.vac_capacity = 0

        for car_index, car in enumerate(cars_list):
            self.cars[car_index] = car
            self.vac_capacity += car.batteryCapacity

        self.number_of_cars = len(self.cars)

    def max_charge_power_calculator(self, charging_period):
        """
        Returns a dictionary "connections"
            keys: charger labels
            values: max charging power input to the connected car
        """

        connections = {}
        for key, charger in self.chargers.items():
            if charger.hosted_car:
                car = charger.hosted_car
                battery_depth_of_discharge = (1 - charger.soc) * car.batteryCapacity  # max_charging_power_kw-sec

                charger_limit = charger.kW
                car_limit = battery_depth_of_discharge / charging_period

                connections[key] = min(charger_limit, car_limit)

        return connections


## Car

In [23]:
class Car:

    def __init__(self, car_name, battery_capacity):
        self.car_name = car_name
        self.batteryCapacity = battery_capacity * 3600

    def charge(self, soc, charge_period, charge_power):
        return soc + charge_power * charge_period / self.batteryCapacity


## ChargingStation

In [7]:
class ChargingStation:

    def __init__(self, max_charging_power_kw, hosted_car=None, soc=None):
        self.max_charging_power_kw = max_charging_power_kw

        if hosted_car:
            self.hosted_car = hosted_car
            self.plugged = True
            self.soc = soc
        else:
            self.plugged = False

    def plug(self, car):
        self.hosted_car = car
        self.plugged = True

    def unplug(self):
        self.hosted_car = None
        self.plugged = False

    def charge(self, charge_period, charge_power):

        if self.plugged == 1:
            self.soc = self.hosted_car.charge(self.soc, charge_period, charge_power)
        else:
            print("Charge decision cannot be implemented")


## Input parser

In [41]:
class InputConfigParser:
    
    def __init__(self, input_config_file, input_config_mqtt, model_name):
        self.model_name = model_name
        self.model_variables = {'Feasible_ESS_Decisions': {'type': 'Set', 'indexing': 'None'}, 'Feasible_VAC_Decisions': {'type': 'Set', 'indexing': 'None'}, 'Value_Index': {'type': 'Set', 'indexing': 'None'}, 'Value': {'type': 'Param', 'indexing': 'index'}, 'P_PV_Forecast': {'type': 'Param', 'indexing': 'None'}, 'Initial_ESS_SoC': {'type': 'Param', 'indexing': 'index'}, 'Initial_VAC_SoC': {'type': 'Param', 'indexing': 'index'}, 'Number_of_Parked_Cars': {'type': 'Param', 'indexing': 'None'}, 'Unit_Consumption_Assumption': {'type': 'Param', 'indexing': 'None'}, 'Unit_Drop_Penalty': {'type': 'Param', 'indexing': 'None'}, 'ESS_Capacity': {'type': 'Param', 'indexing': 'None'}, 'VAC_Capacity': {'type': 'Param', 'indexing': 'None'}, 'Behavior_Model_Index': {'type': 'Set', 'indexing': 'None'}, 'Behavior_Model': {'type': 'Param', 'indexing': 'None'}, 'dT': {'type': 'Param', 'indexing': 'None'}, 'Decision': {'type': 'Var', 'indexing': 'index'}, 'P_ESS': {'type': 'Var', 'indexing': 'None'}, 'P_VAC': {'type': 'Var', 'indexing': 'None'}, 'P_PV': {'type': 'Var', 'indexing': 'index'}, 'P_GRID': {'type': 'Var', 'indexing': 'None'}}
        self.input_config_file = input_config_file
        self.input_config_mqtt = input_config_mqtt
        self.mqtt_params = {}
        self.generic_names = []
        self.generic_file_names = []
        
        self.defined_prediction_names = []
        self.defined_non_prediction_names = ["P_PV", "P_PV_Forecast"] #New
        self.defined_external_names = ["SoC_Value"]
        self.prediction_names = []
        self.non_prediction_names = []
        self.external_names = []
        
        self.optimization_params = self.extract_optimization_values()
        
        # Added
        self.car_park = None
        
    def add_name_to_list(self, key):
        if key in self.defined_prediction_names:
            self.prediction_names.append(key)
        elif key in self.defined_non_prediction_names:
            self.non_prediction_names.append(key)
        elif key in self.defined_external_names:
            self.external_names.append(key)
        else:
            self.generic_names.append(key)
        
    def generate_car_classes(self, cars):
        cars_list = []
        for car_name, car_detail in cars.items():
            battery_capacity = car_detail.get("Battery_Capacity_kWh", None)
            assert battery_capacity, f"Incorrect input: Battery_Capacity_kWh missing for car: {car_name}"
            cars_list.append(Car(car_name, battery_capacity))
        return cars_list

    def generate_charger_classes(self, chargers):
        chargers_list = []
        for charger_name, charger_detail in chargers.items():
            {'Max_Charging_Power_kW': 6, 'Hosted_Car': 'Car1', 'SoC': 0.5}
            max_charging_power_kw = charger_detail.get("Max_Charging_Power_kW", None)
            hosted_car = charger_detail.get("Hosted_Car", None)
            soc = charger_detail.get("SoC", None)
            assert max_charging_power_kw, f"Incorrect input: Max_Charging_Power_kW missing for charger: {charger_name}"
            chargers_list.append(ChargingStation(max_charging_power_kw, hosted_car, soc))
        return chargers_list
    
    def extract_optimization_values(self):
        data = {}
        for input_config in [self.input_config_file, self.input_config_mqtt]:
            for k, v in input_config.items():
                if isinstance(v, dict):
                    for k1, v1 in v.items():
                        if k1 == Constants.meta:
                            for k2, v2 in v1.items():
                                try:
                                    v2 = float(v2)
                                except ValueError:
                                    pass
                                if isinstance(v2, float) and v2.is_integer():
                                    v2 = int(v2)
                                if k2 in self.model_variables.keys():
                                    indexing = self.model_variables[k2]["indexing"]
                                    if indexing == "index":
                                        data[k2] = {int(0): v2}
                                    elif indexing == "None":
                                        data[k2] = {None: v2}
                                else:
                                    data[k2] = {None: v2}
                        elif k1 == Constants.SoC_Value and isinstance(v1, int):
                            indexing = self.model_variables[Constants.SoC_Value]["indexing"]
                            if indexing == "index":
                                data[Constants.SoC_Value] = {int(0): float(v1 / 100)}
                            elif indexing == "None":
                                data[Constants.SoC_Value] = {None: float(v1 / 100)}
                        elif isinstance(v1, list):
                            self.add_name_to_list(k1)
                        elif k == "generic" and not isinstance(v1, dict):
                            logger.debug("Generic single value")
                            try:
                                v1 = float(v1)
                            except ValueError:
                                pass
                            if isinstance(v1, float) and v1.is_integer():
                                v1 = int(v1)
                            data[k1] = {None: v1}
                        elif k == "PROFEV":
                            if isinstance(v1, dict):
                                if k1 == Constants.CarPark:
                                    chargers = v1.get("Charging_Station", None)
                                    cars = v1.get("Cars", None)
                                    assert chargers, "Incorrect input: Charging_Station missing in CarPark"
                                    assert cars, "Incorrect input: Cars missing in CarPark"
                                    chargers = dict(chargers)
                                    cars = dict(cars)
                                    chargers_list = self.generate_charger_classes(chargers)
                                    cars_list = self.generate_car_classes(cars)
                                    self.car_park = CarPark(chargers_list, cars_list)
                                    
                                    data["Number_of_Parked_Cars"] = { None: self.car_park.number_of_cars }
                                    data["VAC_Capacity"] = { None: self.car_park.vac_capacity }
                                    
                                if k1 == Constants.Uncertainty:
                                    print(v1)
                                    pass
                            else:
                                try:
                                    v1 = float(v1)
                                except:
                                    pass
                                if isinstance(v1, float) and v1.is_integer():
                                    v1 = int(v1)
                                data[k1] = { None: v1 }
                        
        pprint.pprint(data, indent=4)
        return data
    
InputConfigParser(input_config_file, input_config_mqtt, model_name)

{'Plugged_Time': {'mu': 18.76, 'sigma': 1.3}, 'Unplugged_Time': {'mu': 7.32, 'sigma': 0.78}, 'ESS_States': {'Min': 0, 'Max': 100, 'Steps': 10}, 'VAC_States': {'Min': 0, 'Max': 100, 'Steps': 2.5}}
{   'ESS_Capacity': {None: 2430},
    'ESS_Max_Charge_Power': {None: 0.62},
    'ESS_Max_Discharge_Power': {None: 0.62},
    'ESS_Max_SoC': {None: 1},
    'ESS_Min_SoC': {None: 0.2},
    'Max_Voltage_Drop': {None: 1.2315135},
    'Min_Voltage_Drop': {None: 1.0246457},
    'Number_of_Parked_Cars': {None: 7},
    'PV_Inv_Max_Power': {None: 3501},
    'P_Grid_Max_Export_Power': {None: 10},
    'Q_Grid_Max_Export_Power': {None: 10},
    'Unit_Consumption_Assumption': {None: 2.5},
    'Unit_Drop_Penalty': {None: 1},
    'VAC_Capacity': {None: 756000},
    'pf_Load': {None: 1}}
